# Notebook pour créer des courbes de lumière
En utilisant les fonctions de calibration et de photométrie disponibles, on va créer simplement une courbe de lumière à partir des images d'un dossier (correspondant à une cible).

In [ ]:
import os, time
from du_astro_utils import calibration, photometry, utils
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.table import Table
from astropy.stats import sigma_clipped_stats
from astropy.timeseries import TimeSeries
from scipy.ndimage import median_filter

In [ ]:
target_type = "exoplanet"
if target_type == "asteroid":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_ASTER)
elif target_type == "exoplanet":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_EXPLTS)
elif target_type == "variable_star":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_VARSTARS)
os.listdir(data_dir)

In [ ]:
for ddir in os.listdir(data_dir):
    subdata_dir = os.path.join(data_dir, ddir)
    if os.path.isdir(subdata_dir):
        list_fits = [im for im in sorted(os.listdir(subdata_dir)) if ".fits" in im]
        list_fits = sorted(list_fits)
        print(subdata_dir, len(list_fits))

In [ ]:
achoice = os.listdir(data_dir)[0]
reduced = True
aligned = True
subdata_dir = os.path.join(data_dir, achoice)
if reduced:
    subdata_dir = os.path.join(subdata_dir, "REDUCED")
if aligned:
    subdata_dir = os.path.join(subdata_dir, "aligned")
list_fits = [im for im in sorted(os.listdir(subdata_dir)) if ".fits" in os.path.splitext(im)[-1]]
list_fits = sorted(list_fits)

In [ ]:
list_fits

In [ ]:
rerun = True
use_calib = False
use_sextractor = True
test_mode = False  # Only runs on a few images, prints the tables at each step and does not write files.
sex_photom = False
write_res = True

if rerun or test_mode:
    from tqdm import tqdm
    from astropy.timeseries import TimeSeries
    from astropy.table import vstack
    from astropy.wcs import WCS
    from astropy.wcs.utils import skycoord_to_pixel
    from astropy.coordinates import Angle, SkyCoord
    from astropy.time import Time

    import warnings
    from astropy.utils.exceptions import AstropyWarning
    from astropy.coordinates.name_resolve import NameResolveError

    warnings.simplefilter("ignore", category=AstropyWarning)

    import logging

    logger = logging.getLogger()
    logger.setLevel(logging.CRITICAL)

    if test_mode:
        list_fits = list_fits[:10]
    for loc, scimage in enumerate(tqdm(list_fits)):
        fits_sci_image = os.path.join(subdata_dir, scimage)
        fits_sci_image = os.path.abspath(fits_sci_image)
        if os.path.isfile(fits_sci_image):
            # Get image directory, failename and extension
            sc_im_dir = os.path.abspath(os.path.dirname(fits_sci_image))
            sc_im_name, sc_im_ext = os.path.splitext(os.path.basename(fits_sci_image))

            if use_calib:
                bias_dir, darks_dir, flats_dir = utils.get_calib_dirs_photometry(fits_sci_image)

                # Get information from FITS header
                sc_date, sc_scope, sc_cam, sc_filter, sc_expos, sc_x, sc_y = calibration.get_infos_from_image(fits_sci_image, verbose=False)
                # print(sc_date, sc_scope, sc_cam, sc_filter, sc_expos, sc_x, sc_y)

                # Run calibration
                dico_calib = calibration.reduce_sci_image(fits_sci_image, darks_dir, flats_dir, path_to_bias_dir="", use_bias=False, override_date_check=True, max_days=7, speedup=True, verbose=False, write_tmp=True)
                red_sci_image = dico_calib["path"]
            else:
                red_sci_image = fits_sci_image

            # Analysis
            # hdu = fits.open(red_sci_image)[0]
            im_dir = os.path.abspath(os.path.dirname(red_sci_image))
            im_name, im_ext = os.path.splitext(os.path.basename(red_sci_image))
            with fits.open(red_sci_image) as hdul:
                hdu = hdul[0]
                wcs = WCS(hdu.header)
                epoch = Time(hdu.header.get("DATE-OBS"), format="isot")

                # Get the target position
                target = hdu.header.get("OBJECT")
                if not target_type == "asteroid":
                    try:
                        target_coords = SkyCoord.from_name(target)
                    except NameResolveError:
                        target = target.split("-")[-1]  # Quick fix for the case '330-Adalberta' and similar.
                        target_coords = SkyCoord.from_name(target)
                    target_x, target_y = skycoord_to_pixel(target_coords, wcs=wcs)
                    if test_mode:
                        print(target_x, target_y)
            if loc == 0 or test_mode or sex_photom:
                if use_sextractor:
                    sex_cmd = f"source-extractor -c default.sex {red_sci_image} -CATALOG_NAME tmp_sources.cat -CATALOG_TYPE FITS_1.0 -GAIN 0.932 -VERBOSE_TYPE QUIET"
                    os.system(sex_cmd)
                    cat_tab = Table.read("tmp_sources.cat")
                    cat_tab.rename_column("X_IMAGE", "xcentroid")
                    cat_tab.rename_column("Y_IMAGE", "ycentroid")
                    _, fwhm, _ = sigma_clipped_stats(cat_tab["FWHM_IMAGE"])
                    sex_coords = SkyCoord(ra=cat_tab["ALPHA_J2000"], dec=cat_tab["DELTA_J2000"], unit="deg", obstime=epoch)
                    # dist_to_target = np.power(cat_tab["xcentroid"]-target_x, 2) + np.power(cat_tab["ycentroid"]-target_y, 2)
                    dist_to_target = sex_coords.separation(target_coords)
                    id_target = np.nanargmin(dist_to_target)
                    if loc == 0:
                        refs_ids = np.argsort(np.abs(cat_tab["FLUX_AUTO"] - cat_tab[id_target]["FLUX_AUTO"]))[1:6]
                        ref_coords = SkyCoord(ra=cat_tab[refs_ids]["ALPHA_J2000"], dec=cat_tab[refs_ids]["DELTA_J2000"], unit="deg", obstime=epoch)
                    else:
                        refs_ids = np.array([np.nanargmin(sex_coords.separation(refradec)) for refradec in ref_coords]).flatten()

                    if sex_photom and not (target_type == "asteroid"):
                        if test_mode:
                            print(id_target, dist_to_target[id_target])
                        # if np.sqrt(dist_to_target[id_target]) < 2*fwhm :
                        sex_target_table = cat_tab[id_target]
                        sex_ref_table = cat_tab[refs_ids]
                        sex_target_ts = TimeSeries(time=[epoch], data=sex_target_table)
                        for i, val in enumerate(sex_ref_table["FLUX_AUTO"]):
                            sex_target_ts.add_column(val, name=f"FLUX_REF{i+1}")
                        if test_mode:
                            print(sex_target_ts)
                        if loc == 0:
                            concat_sex = sex_target_ts
                        else:
                            try:
                                concat_sex = vstack([concat_sex, sex_target_ts])
                            except ValueError:
                                pass
                else:
                    try:
                        sources = photometry.detect_sources(red_sci_image, detection_fwhm=10, verbose=False)
                        fwhm = photometry.get_fwhm(red_sci_image, sources)
                    except RuntimeError:
                        fwhm = 10
            if target_type == "asteroid":
                phot_target_table = photometry.query_named_sso_photometry(red_sci_image, fwhm, verbose=False)
            elif test_mode or not (sex_photom):
                phot_target_table = photometry.apert_photometry_target(red_sci_image, fwhm, verbose=False)
                ref_target_table = photometry.apert_photometry(red_sci_image, cat_tab[refs_ids], fwhm)
                for i, (ap, bgsub) in enumerate(zip(ref_target_table["aperture_sum"], ref_target_table["aper_sum_bkgsub"])):
                    phot_target_table.add_column(ap, name=f"aperture_sum_REF{i+1}")
                    phot_target_table.add_column(bgsub, name=f"aper_sum_bkgsub_REF{i+1}")
            if loc == 0:
                concat_ts = phot_target_table
            else:
                try:
                    concat_ts = vstack([concat_ts, phot_target_table])
                except ValueError:
                    pass
    if not (test_mode) and sex_photom:
        concat_ts = concat_sex

    if write_res and not (test_mode):
        try:
            if use_calib:
                if aligned:
                    concat_ts.write(f"{subdata_dir.split('-')[-1].split('/')[0]}_aligned_CAL.fits", format="fits", overwrite=True)
                else:
                    concat_ts.write(f"{subdata_dir.split('-')[-1]}_CAL.fits", format="fits", overwrite=True)
            else:
                if aligned:
                    concat_ts.write(f"{subdata_dir.split('-')[-1].split('/')[0]}_aligned.fits", format="fits", overwrite=True)
                else:
                    concat_ts.write(f"{subdata_dir.split('-')[-1]}.fits", format="fits", overwrite=True)
        except OSError:
            time.sleep(5)
            if use_calib:
                if aligned:
                    concat_ts.write(f"{subdata_dir.split('-')[-1].split('/')[0]}_aligned_CAL.fits", format="fits", overwrite=True)
                else:
                    concat_ts.write(f"{subdata_dir.split('-')[-1]}_CAL.fits", format="fits", overwrite=True)
            else:
                if aligned:
                    concat_ts.write(f"{subdata_dir.split('-')[-1].split('/')[0]}_aligned.fits", format="fits", overwrite=True)
                else:
                    concat_ts.write(f"{subdata_dir.split('-')[-1]}.fits", format="fits", overwrite=True)
    elif test_mode:
        concat_ts.pprint_all()
        concat_sex.pprint_all()
        # plt.errorbar(phot_target_table["Epoch"], phot_target_table["aper_sum_bkgsub"], yerr=phot_target_table["noise"])
else:
    if use_calib:
        if aligned:
            concat_ts = Table.read(f"{subdata_dir.split('-')[-1].split('/')[0]}_aligned_CAL.fits", format="fits")
        else:
            concat_ts = Table.read(f"{subdata_dir.split('-')[-1]}_CAL.fits", format="fits")
    else:
        if aligned:
            concat_ts = Table.read(f"{subdata_dir.split('-')[-1].split('/')[0]}_aligned.fits", format="fits")
        else:
            concat_ts = Table.read(f"{subdata_dir.split('-')[-1]}.fits", format="fits")

In [ ]:
concat_ts

In [ ]:
concat_ts.time.mjd

In [ ]:
from astropy.stats import sigma_clipped_stats


def detrend_fun(x, a, b, c, d):
    return a * x**3 + b * x**2 + c * x + d


from scipy.optimize import curve_fit

if sex_photom:
    mean, med, sig = sigma_clipped_stats(concat_ts["FLUX_AUTO"])

    sel = np.logical_and(concat_ts["FLUX_AUTO"] > med - 7 * sig, concat_ts["FLUX_AUTO"] < med + 7 * sig)
    plt.errorbar(concat_ts[sel].time.mjd, concat_ts[sel]["FLUX_AUTO"] / med.value, yerr=concat_ts[sel]["FLUXERR_AUTO"] / med.value, fmt=".", c="k")
    for i in range(1, 6):
        mean, med, sig = sigma_clipped_stats(concat_ts[f"FLUX_REF{i}"])
        sel = np.logical_and(concat_ts[f"FLUX_REF{i}"] > med - 7 * sig, concat_ts[f"FLUX_REF{i}"] < med + 7 * sig)
        plt.scatter(concat_ts[sel].time.datetime64, concat_ts[sel][f"FLUX_REF{i}"] / med, marker="+", s=2)
        plt.ylim(0.8, 1.2)
    detrend_coeffs = curve_fit(detrend_fun, concat_ts[sel].time.mjd, concat_ts[sel]["FLUX_AUTO"], sigma=concat_ts[sel]["FLUXERR_AUTO"])[0]
    plt.plot(concat_ts[sel].time.mjd, detrend_fun(concat_ts[sel].time.mjd, *detrend_coeffs) / med.value)
else:
    mean, med, sig = sigma_clipped_stats(concat_ts["aper_sum_bkgsub"])
    sel = np.logical_and(concat_ts["aper_sum_bkgsub"] > med - 7 * sig, concat_ts["aper_sum_bkgsub"] < med + 7 * sig)
    plt.scatter(concat_ts[sel].time.mjd, concat_ts[sel]["aper_sum_bkgsub"] / med)
    plt.tick_params(axis="x", labelrotation=90)
    detrend_coeffs = curve_fit(detrend_fun, concat_ts[sel].time.mjd, concat_ts[sel]["aper_sum_bkgsub"].value)[0]
    plt.plot(concat_ts[sel].time.mjd, detrend_fun(concat_ts[sel].time.mjd, *detrend_coeffs) / med)

In [ ]:
detrend_coeffs

In [ ]:
detrend = detrend_fun(concat_ts[sel].time.mjd, *detrend_coeffs)
plt.errorbar(concat_ts[sel].time.mjd, concat_ts[sel]["aper_sum_bkgsub"] / detrend, yerr=concat_ts[sel]["noise"] / detrend, fmt=".")

In [ ]:
kal795 = Table.read("1_CAL.fits", format="fits")  # Table.read("Kalliope_CAL_795sur991.fits", format="fits")
kal795

In [ ]:
mean, med, sig = sigma_clipped_stats(kal795["aper_sum_bkgsub"])
sel = np.logical_and(kal795["aper_sum_bkgsub"] > max(0.0, med - 5 * sig), kal795["aper_sum_bkgsub"] < med + 5 * sig)
plt.scatter(kal795[sel]["time"][:, 1], kal795[sel]["aper_sum_bkgsub"])
plt.tick_params(axis="x", labelrotation=90)

In [ ]:
mean, med, sig = sigma_clipped_stats(kal795["aperture_sum"])
sel = np.logical_and(kal795["aperture_sum"] > max(0.0, med - 5 * sig), kal795["aperture_sum"] < med + 5 * sig)
plt.scatter(kal795[sel]["time"][:, 1], kal795[sel]["aperture_sum"])
plt.tick_params(axis="x", labelrotation=90)

In [ ]:
kal795[sel]["aperture_sum"].shape

In [ ]:
n_per_bin = 10
ts = []
vals = []
sigs = []
for ix in range(len(kal795[sel]["aperture_sum"]) // n_per_bin):
    pts = kal795["aperture_sum"][ix * n_per_bin : (ix + 1) * n_per_bin]
    mean, med, sig = sigma_clipped_stats(pts)
    t = np.median(kal795["time"][ix * n_per_bin : (ix + 1) * n_per_bin, 1])
    ts.append(t)
    vals.append(med)
    sigs.append(sig)
plt.errorbar(ts, vals, yerr=sigs, fmt=".")

In [ ]:
n_per_bin = 10
ts = []
vals = []
sigs = []
for ix in range(len(kal795[sel]["aper_sum_bkgsub"]) // n_per_bin):
    pts = kal795["aper_sum_bkgsub"][ix * n_per_bin : (ix + 1) * n_per_bin]
    mean, med, sig = sigma_clipped_stats(pts)
    t = np.median(kal795["time"][ix * n_per_bin : (ix + 1) * n_per_bin, 1])
    ts.append(t)
    vals.append(med)
    sigs.append(sig)
plt.errorbar(ts, vals, yerr=sigs, fmt=".")